# Biol 359A | Linear Regression
### Spring 2024, Week 4
Objectives:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, Dropdown, Checkbox, HBox, VBox, Button, Output, Layout, IntSlider
import seaborn as sns
from scipy.stats import pearsonr
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

In [ ]:
! rm -r week4_linearregression/
! git clone https://github.com/BIOL359A-FoundationsOfQBio-Spr24/week4_linearregression.git
! cp -r week4_linearregression/* .
! ls

# Introduction to Linear Regression

Linear regression assumes a linear relationship between the input variables (predictors) and the single output variable (response). When the input variable is one, the method is referred to as simple linear regression. When there are multiple input variables, it is called multiple linear regression.

## Key Terms in Linear Regression

- **Predictor Variables (Independent variables)**: These are the variables/input that predict or affect the outcome. They are used as the basis to predict the response variable.
- **Response Variable (Dependent variable)**: This is the outcome variable that the model is trying to predict, using predictor variables.
- **Coefficients**: These are the weights assigned to the predictor variables. They signify the change in the response variable for a one-unit change in the predictor.
- **Intercept**: This is the expected mean value of the response variable when all the predictors are equal to zero.
- **Residual Error**: This is the difference between the observed values and the values predicted by the model. It represents the error in the predictions.



### Simple Linear Regression

Simple linear regression is a statistical method that allows us to summarize and study relationships between two continuous (quantitative) variables:
1. The variable x is the predictor, or independent variable.
2. The variable y is the response, or dependent variable.

Simple linear regression uses one independent variable to explain or predict the outcome of the dependent variable Y, based on the linear relationship between X and Y.

### Coefficient of Determination

The __Coefficient of Determination__, or $R^2$, conceptually represents the "amount of variance in y is explained by x" and can be thought of as how well the model fits the training data. This is a seperate concept to the Pearson Correlation Coefficient, $\rho$, which in the cases of a __strictly linear regression__, (regression which is linear with respect to both coefficients and parameters) $R^2 = \rho^2$. $R^2$ is calculated as follows:

$$ R^2 = \frac{SSR}{SST} = 1 - \frac{SSE}{SST}$$

Where 
- SSE = error of sum of squares
- SSR = regression sum of squares
- SST = total sum of squares

You can also calculate the adjusted $R^2$ which accounts for the model size and parameters. This way, you can compare the adjusted $R^2$ of simple models to the adjusted $R^2$ of complex models, which you can't necessarily do if you just use  $R^2$. 

$$R^2_{adj} = 1 - \frac{\frac{SSE}{n-k-1}}{\frac{SST}{n - 1}}$$

Where 
- n = number of data points
- k number of independent variables

Generally speaking, $R^2$ is used to characterize the fit of a model, where as $\rho$ is used to characterize the relationship between two variables. 

We will perform a simple linear regression between the response variable, y, and the feature, x. 


ASSIGNMENT QUESTIONS:
- Please answer questions 6 and 7 in the assignment.

### Introduction to the Allen Institute for Cell Science Cell Feature Dataset

The [Allen Institute for Cell Science Cell Feature Dataset](https://www.allencell.org/data-downloading.html#DownloadFeatureData) was published with [Viana et al. 2023](https://www.nature.com/articles/s41586-022-05563-7). These data are visualized in the [Cell Feature Explorer](https://cfe.allencell.org/).

**Dataset Features**:
- **`CellId`**: A unique identifier for each cell in the dataset.
- **`Nuclear Volume [fL]`**: The volume of the cell's nucleus measured in femtoliters (fL).
- **`Cellular Volume [fL]`**: The total volume of the cell measured in femtoliters (fL).
- **`Nuclear Surface Area [um^2]`**: The surface area of the cell’s nucleus measured in square micrometers (um^2).
- **`Cellular Surface Area [um^2]`**: The total surface area of the cell measured in square micrometers (um^2).
- **`Radial Proximity [unitless]`**: A unitless measure that quantifies the cell's position relative to the center of the organ or tissue.
- **`Apical Proximity [unitless]`**: Another unitless measure that describes how close the cell is to the apical surface of the tissue.

In [ ]:
# Load the dataset
data = pd.read_csv('data/aics_cfe_cell-features_v1.8.csv')

# Print the shape of the data (rows, columns)
print(data.shape)

# Display the first few rows of data
data.head()

ASSIGNMENT QUESTION:
- Please complete question 8 in the assignment.

The interactive visualization below allows you to select two different features from the Allen dataset and plot them against each other in a scatter plot with the simple linear regression line.

The regression outputs are:
- **beta1**: This is the slope of the regression line, representing the change in the dependent variable (y) for each unit change in the explanatory variable (x).
- **beta0**: This is the intercept of the regression line, representing the value of y when x is 0.
- **R² (R-squared)**: This statistic measures the proportion of variance in the dependent variable that is predictable from the explanatory variable.
- **Pearson's Correlation Coefficient**: This measures the linear correlation between two variables, ranging from -1 to 1, where 1 means a perfect positive correlation, -1 means a perfect negative correlation, and 0 means no linear correlation.


In [ ]:
def plot_regression(x_feature, y_feature):
    # Filtering out the CellID and setting the data for regression
    x = data[x_feature].values.reshape(-1, 1)
    y = data[y_feature].values.reshape(-1, 1)
    
    # Fitting the linear regression model
    model = LinearRegression()
    model.fit(x, y)
    y_pred = model.predict(x)
    
    # Calculating regression statistics
    beta1 = model.coef_[0][0]
    beta0 = model.intercept_[0]
    r_squared = model.score(x, y)
    corr_coef, _ = pearsonr(data[x_feature], data[y_feature])
    
    # Plotting the results
    plt.figure(figsize=(10, 6))
    plt.scatter(x, y, color='blue', label='Data points')
    plt.plot(x, y_pred, color='red', label='Regression line')
    plt.xlabel(x_feature)
    plt.ylabel(y_feature)
    plt.title(f'Scatter plot of {x_feature} vs {y_feature}')
    plt.legend()
    plt.grid(True)
    plt.show()
    
    # Displaying the regression results
    print(f"beta1 (Slope): {beta1:.4f}")
    print(f"beta0 (Intercept): {beta0:.4f}")
    print(f"R-squared: {r_squared:.4f}")
    print(f"Pearson's Correlation Coefficient: {corr_coef:.4f}")

# Creating dropdowns for feature selection
features = [col for col in data.columns if col != 'CellId']  # Exclude 'CellID'
_ = interact(plot_regression, x_feature=Dropdown(options=features), y_feature=Dropdown(options=features))


DISCUSSION QUESTION:
- What is the relationship between R^2 and Pearson's Correlation Coefficient in the above plot? Why?

### Statistical tests and $\beta$ coefficients

When performing linear regression, each regression coefficient ($\beta$) represents the expected change in the dependent variable for a one-unit change in the corresponding independent variable, holding all other variables constant. To understand the statistical significance of these coefficients, you can conduct hypothesis tests.

The null hypothesis typically states that the coefficient in question, ($\beta_i$)​, has no effect on the dependent variable, implying $\beta_i=0$. To test this hypothesis, you use the t-statistic, which is calculated by dividing the estimated coefficient by its standard error. The resulting t-value is then compared against a critical value from the t-distribution, considering the desired level of confidence and the degrees of freedom in your data.

When the independent variable in the regression is binary or boolean (i.e., it only takes two values), the regression model simplifies the comparison between two groups. In this scenario, testing $\beta_i=0$ is analogous to performing an independent samples t-test between these two groups. The t-test compares the means of two groups and tests whether they are statistically different from each other. In the regression context, the coefficient $\beta_i$​ for a binary variable measures the difference in the mean of the dependent variable between the two groups defined by the binary variable.

To solidify this concept, we will use the Wisconsin Diagnostic Breast Cancer dataset once again and use the boolean dianosis status of each cell to try to predict the other cell features. A reminder on the dataset:
```
	Features are computed from a digitized image of a fine needle
	aspirate (FNA) of a breast mass.  They describe
	characteristics of the cell nuclei present in the image.
	A few of the images can be found at
	http://www.cs.wisc.edu/~street/images/

	Separating plane described above was obtained using
	Multisurface Method-Tree (MSM-T) [K. P. Bennett, "Decision Tree
	Construction Via Linear Programming." Proceedings of the 4th
	Midwest Artificial Intelligence and Cognitive Science Society,
	pp. 97-101, 1992], a classification method which uses linear
	programming to construct a decision tree.  Relevant features
	were selected using an exhaustive search in the space of 1-4
	features and 1-3 separating planes.

	The actual linear program used to obtain the separating plane
	in the 3-dimensional space is that described in:
	[K. P. Bennett and O. L. Mangasarian: "Robust Linear
	Programming Discrimination of Two Linearly Inseparable Sets",
	Optimization Methods and Software 1, 1992, 23-34].
    
    Source:
    W.N. Street, W.H. Wolberg and O.L. Mangasarian 
	Nuclear feature extraction for breast tumor diagnosis.
	IS&T/SPIE 1993 International Symposium on Electronic Imaging: Science
	and Technology, volume 1905, pages 861-870, San Jose, CA, 1993.
```

What do all the column names mean?

- ID number
- Diagnosis (M = malignant, B = benign)

Ten real-valued features are computed for each cell nucleus:

- radius (mean of distances from center to points on the perimeter)
- texture (standard deviation of gray-scale values)
- perimeter
- area
- smoothness (local variation in radius lengths)
- compactness (perimeter^2 / area - 1.0)
- concavity (severity of concave portions of the contour)
- concave points (number of concave portions of the contour)
- symmetry 
- fractal dimension ("coastline approximation" - 1) - a measure of "complexity" of a 2D image.


Cateogory Distribution: 357 benign, 212 malignant

In [ ]:
import clean_data

original_cancer_dataset = clean_data.generate_clean_dataframe()
cancer_dataset = original_cancer_dataset.reset_index()
cancer_dataset.drop("ID", axis=1, inplace=True)
cancer_dataset

The visuzalisation below allows you to select a feature and see the distribution of that feature for malignant vs benign cells. It then plots diagnosis status as the independent variable and the selected feature as the dependent variable and performs simple linear regression.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from scipy.stats import pearsonr, t
from ipywidgets import interact, Dropdown


# Mapping the 'diagnosis' to a binary variable for regression purposes
cancer_dataset['diagnosis_code'] = cancer_dataset['diagnosis'].map({'M': 1, 'B': 0})

def interactive_feature_visualization(feature):
    # Plotting the distribution of the feature for malignant and benign tumors
    malignant = cancer_dataset[cancer_dataset['diagnosis'] == 'M'][feature]
    benign = cancer_dataset[cancer_dataset['diagnosis'] == 'B'][feature]

    plt.figure(figsize=(18, 6))

    plt.subplot(1, 3, 1)
    sns.histplot(malignant, color='red', kde=True, stat="density", linewidth=0, label='Malignant')
    sns.histplot(benign, color='blue', kde=True, stat="density", linewidth=0, label='Benign')
    plt.title(f'Distribution of {feature} for Malignant vs Benign')
    plt.legend()

    plt.subplot(1, 3, 2)
    sns.boxplot(x='diagnosis', y=feature, data=cancer_dataset)
    plt.title(f'Box Plot of {feature} by Diagnosis')

    # Regression Analysis
    x = cancer_dataset[['diagnosis_code']].values  # Independent variable
    y = cancer_dataset[feature].values  # Dependent variable

    model = LinearRegression()
    model.fit(x, y)
    y_pred = model.predict(x)

    # Scatter plot with regression line
    plt.subplot(1, 3, 3)
    plt.scatter(x, y, color='green', label='Actual data')
    plt.plot(x, y_pred, color='black', linewidth=2, label='Regression line')
    plt.title(f'Scatter Plot and Regression Line for {feature}')
    plt.xlabel('Diagnosis Code (0=Benign, 1=Malignant)')
    plt.ylabel(feature)
    plt.xticks([0, 1], ['Benign', 'Malignant'])
    plt.legend()

    plt.show()

    # Calculating regression statistics
    beta1 = model.coef_[0]
    beta0 = model.intercept_
    r_squared = model.score(x, y)
    n = len(y)
    residual_std_error = np.sqrt(np.sum((y - y_pred) ** 2) / (n - 2))
    corr_coef, _ = pearsonr(cancer_dataset['diagnosis_code'], y)
    std_error_beta1 = residual_std_error / np.sqrt(np.sum((x - np.mean(x)) ** 2))
    t_statistic = beta1 / std_error_beta1
    p_value = (1 - t.cdf(np.abs(t_statistic), df=n-2)) * 2  # two-tailed p-value

    print(f"Linear Regression Analysis for {feature} based on Diagnosis:")
    print(f"beta1 (Effect of Malignant over Benign): {beta1:.4f}")
    print(f"beta0 (Intercept, Effect of Benign): {beta0:.4f}")
    print(f"R-squared: {r_squared:.4f}")
    print(f"Pearson's Correlation Coefficient: {corr_coef:.4f}")
    print(f"t-statistic for beta1: {t_statistic:.4f}")
    print(f"p-value for beta1: {p_value:.4f}")

# Creating dropdown for feature selection, excluding 'diagnosis'
features = [col for col in cancer_dataset.columns if col not in ['diagnosis', 'diagnosis_code']]
_ = interact(interactive_feature_visualization, feature=Dropdown(options=features))

DISCUSSION QUESTION:
- How does the R-squared value relate to the Pearson's Correlation Coefficient? Why?

## Multiple Linear Regression

We have other variables that we can use to explore the relationships in this dataset. We can use multiple linear regression to include more variables, where we are drawing a multi-dimensional shape, rather than simply a line, as our model. 

The visualization below allows you to perform multiple linear regression. Select a dependent (or response variable) using the drop down and select independent variables using the check boxes. The plot shown is a parity plot, which shows the true value of the specified dependent variable plotted against the model-predicted value of the specified dependent variable. Selecting one independent variable will perform simple linear regression, but selecting more independent variables will perform multiple linear regression. The dotted black line is the parity line. It depicts where all the data would be if the model perfectly predicted the values for the dependent variable. Try adding multiple independent variables into the regression by checking the boxes and see how it affects the model's ability to predict different dependent variables.

In [ ]:
cancer_dataset['diagnosis_code'] = cancer_dataset['diagnosis'].map({'M': 1, 'B': 0})

# Define plot_output outside the function to ensure it's in the correct scope
plot_output = Output()

def multiple_regression_visualization(response):
    features = [col for col in cancer_dataset.columns if col not in [response, 'diagnosis', 'diagnosis_code']]
    checkboxes = {feature: Checkbox(value=False, description=feature, style={'description_width': 'initial'}) for feature in features}
    checkbox_container = VBox(list(checkboxes.values()), layout=Layout(width='100%', overflow='auto'))
    button = Button(description="Update Regression")
    
    def on_button_clicked(b):
        selected_features = [feat for feat, cb in checkboxes.items() if cb.value]
        perform_regression(response, selected_features)
    
    button.on_click(on_button_clicked)
    
    # Only display widgets and plot_output container together
    display(VBox([checkbox_container, button, plot_output]))

def perform_regression(response, selected_features):
    with plot_output:
        plot_output.clear_output(wait=True)
        if selected_features:
            X = cancer_dataset[selected_features]
            y = cancer_dataset[response]
            
            # Fitting multiple linear regression
            model = LinearRegression()
            model.fit(X, y)
            y_pred = model.predict(X)
            
            plt.figure(figsize=(12, 6))
            # Parity plot
            scatter = plt.scatter(y, y_pred, c=cancer_dataset['diagnosis_code'], cmap=plt.cm.coolwarm, alpha=0.6, edgecolors='w')
            plt.plot([y.min(), y.max()], [y.min(), y.max()], 'k--', lw=2)  # Line of perfect parity
            plt.xlabel('True Values')
            plt.ylabel('Predictions')
            plt.title('Parity Plot of Predictions vs. True Values')
            
            # Creating a custom legend for diagnosis
            legend1 = plt.legend(*scatter.legend_elements(), title="Diagnosis")
            legend1.get_texts()[0].set_text('Benign')
            legend1.get_texts()[1].set_text('Malignant')
            plt.gca().add_artist(legend1)
            
            plt.show()
            
            # Output the regression coefficients and R-squared
            print("Regression coefficients:")
            print("Intercept:", model.intercept_)
            for feature, coef in zip(selected_features, model.coef_):
                print(f"{feature}: {coef}")
            print("R-squared:", r2_score(y, y_pred))
        else:
            print("Please select at least one feature.")

# Creating dropdown for response variable selection
response_features = [col for col in cancer_dataset.columns if col not in ['diagnosis', 'diagnosis_code']]
_ = interact(multiple_regression_visualization, response=Dropdown(options=response_features, description="Select Response Variable:", style={'description_width': 'initial'}))


ASSIGNMENT QUESTION:
- Please answer questions 9 and 10 in the assignment.

DISCUSSION QUESTION:
- Can you compare the significance of impact of each independent variable on the dependent variable using the magnitude of their respective regression coefficients? (Precursor to the next section)

## Normalization

We need to be careful about how we interpret parameters in relation to each other. One method that we can use to compare coefficients is to __standardize__ the features. This will effectively shift the distribution of all of our features to have a Standard Normal ($\mu=0,\sigma^2=1$) distribution (a normal distribution with a mean of 0 and a variance of 1). We can then compare the coefficients to each other, and interpret the magnitude within the model. 

Note: This transformation prevents you from making a direct interpretation of the coefficient as it relates to the actual value, and is now unitless. You should interpret these as being "model units". This transformation also means that the y-intercept will be zero. 

In [ ]:
cancer_dataset['diagnosis_code'] = cancer_dataset['diagnosis'].map({'M': 1, 'B': 0})

plot_output = Output()

def multiple_regression_standardized_visualization(response):
    features = [col for col in cancer_dataset.columns if col not in [response, 'diagnosis', 'diagnosis_code']]
    checkboxes = {feature: Checkbox(value=False, description=feature, style={'description_width': 'initial'}) for feature in features}
    checkbox_container = VBox(list(checkboxes.values()), layout=Layout(width='100%', overflow='auto'))
    button = Button(description="Update Regression")

    def on_button_clicked(b):
        selected_features = [feat for feat, cb in checkboxes.items() if cb.value]
        perform_standardized_regression(response, selected_features)

    button.on_click(on_button_clicked)
    
    display(VBox([checkbox_container, button, plot_output]))

def perform_standardized_regression(response, selected_features):
    with plot_output:
        plot_output.clear_output(wait=True)
        if selected_features:
            X = cancer_dataset[selected_features]
            y = cancer_dataset[response]
            
            # Standardizing both the features and the response
            scaler_X = StandardScaler()
            scaler_y = StandardScaler()
            X_scaled = scaler_X.fit_transform(X)
            y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1)).flatten()
            
            # Fitting multiple linear regression
            model = LinearRegression()
            model.fit(X_scaled, y_scaled)
            
            print("Regression coefficients for fully standardized data:")
            print(f"Intercept (should be close to 0) ~ {model.intercept_:.3}")
            for feature, coef in zip(selected_features, model.coef_):
                print(f"{feature} ~ {coef:.3}")
            print(f"R-squared: {model.score(X_scaled, y_scaled):.4f}")
        else:
            print("Please select at least one feature.")

response_features = [col for col in cancer_dataset.columns if col not in ['diagnosis', 'diagnosis_code']]
_ = interact(multiple_regression_standardized_visualization, response=Dropdown(options=response_features, description="Select Response Variable:", style={'description_width': 'initial'}))


ASSIGNMENT QUESTION:
- Please answer question 11 in the assignment

### Introduction to Feature Engineering

Feature engineering is the process of using domain knowledge to select, modify, or create new features from raw data to increase the predictive power of machine learning algorithms. In the context of regression, this often involves transforming or combining input data to better expose the underlying patterns to the learning algorithm.

#### Polynomial Features in Regression

We are going to generate data from a quadratic equation with some noise, and then we are going to try and fit a new linear model:

$$ y = \beta_0 + \sum_{i=1}^{degrees} \beta_i x^i$$ 

Where we are going to take an original dataset of 1 feature, and generate multiple new features from the original feature. From this equation, if we set `degrees` equal to 1, we are doing a simple linear regression. 

If we set `degrees` equal to 2, we are fitting a MLR model with $x$ and $x^2$ as our features. 
Creating polynomial features is one way that we can do feature engineering.

When creating polynomial features for a regression model, the primary purpose is to add complexity to the model, allowing it to capture more intricate patterns in the data. Polynomial features enable the model to fit non-linear relationships within a linear model framework by considering higher-degree terms of the input features. This can be crucial when the relationship between the independent and dependent variables is not linear but still follows some predictable pattern that higher-degree polynomials can approximate.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score
from ipywidgets import interact, IntSlider, Layout
import seaborn as sns

def generate_data(num_points):
    np.random.seed(0)
    X = 2 - 3 * np.random.normal(0, 1, num_points)
    y = X - 2 * (X ** 2) + np.random.normal(-3, 3, num_points)
    return X, y

def plot_polynomial_regression(num_points=20, degree=2):
    X, y = generate_data(num_points)
    X = X[:, np.newaxis]
    
    polynomial_features = PolynomialFeatures(degree=degree)
    X_poly = polynomial_features.fit_transform(X)
    
    model = LinearRegression()
    model.fit(X_poly, y)
    y_pred = model.predict(X_poly)
    
    plt.figure(figsize=(10, 6))
    plt.scatter(X, y, s=30, marker='o', label="Data Points")
    sorted_zip = sorted(zip(X, y_pred))
    X, y_pred = zip(*sorted_zip)
    plt.plot(X, y_pred, color='r', label=f"Polynomial Degree {degree}")
    plt.xlabel('X')
    plt.ylabel('y')
    plt.title('Polynomial Regression Fit')
    plt.legend()
    plt.grid(True)
    plt.show()
    
    print(f"R-squared for polynomial degree {degree}: {r2_score(y, y_pred):.4f}")

# Interactive widgets with adjusted layout
_ = interact(plot_polynomial_regression, 
         num_points=IntSlider(value=70, min=10, max=100, step=5, description='Number of Data Points', style={'description_width': 'initial'}, layout=Layout(width='500px')),
         degree=IntSlider(value=1, min=1, max=10, step=1, description='Polynomial Degree', style={'description_width': 'initial'}, layout=Layout(width='500px')))


ASSIGNMENT QUESTION:
- Please complete assignment questions 12 and 13

DISCUSSION QUESTION:
- What degree polynomial is sufficient to fit a linear model to these data? Why?